In [ ]:
# Transfer time

In [14]:
import pandas as pd
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [39]:
path = os.getcwd()
file_name_list = []
for file in os.listdir('.'):
    if file.endswith('.csv'):
        file_name_list.append(file)
file_name_list

['Benz.csv', 'Leaf.csv', 'BMW.csv', 'Audi.csv', 'Tesla.csv', 'Bolt.csv']

In [ ]:
for i in file_name_list:
    # Read CSV
    df = pd.read_csv(i, sep=',')
    # Remove useless value '[]'
    df = df[df.time != '[]']
    # Remove missing value
    df = df.dropna(subset=['time'])
    # Transfer the data to integer
    df['time'] = df['time'].apply(lambda x: int(float(x)))
    # Transfer the data to a corresponding time
    df['time'] = df['time'].apply(lambda x: pd.to_datetime(x, unit='s'))
    # Updata the CSV
    df.to_csv(i,index=False)

In [41]:
from afinn import Afinn
af = Afinn()

In [109]:
Bolt = pd.read_csv('Bolt.csv',sep=',')
Leaf = pd.read_csv('Leaf.csv',sep=',')
Tesla = pd.read_csv('Tesla.csv',sep=',')


In [196]:
Tesla = pd.read_csv('Tesla_Eric.csv',sep=',')

In [197]:
Tesla

,comment,comment_id,post_id,post_time,post_title,time,month
0,reminder include price wall connector include ...,epkqq0x,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
1,blue performance model confirm,epkl3uo,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
2,live too base same option same delivery estima...,epklx56,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
3,congrat come pretty soon too sadly can fork ri...,epklo12,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
4,standard range autopilot come well luxury car ...,epklvos,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
...,...,...,...,...,...,...,...
356985,mid when switch then wording where say pricing...,ehj8nkn,aw0ve0,1.551417e+09,It looks like the purchase button has just bee...,2019-03-01,3
356986,promise past will free hardware upgrade buy now,ehj9mwp,aw0ve0,1.551417e+09,It looks like the purchase button has just bee...,2019-03-01,3
356987,way go lower,ehj9efq,aw0ve0,1.551417e+09,It looks like the purchase button has just bee...,2019-03-01,3
356988,maybe think see guy talk how cool traffic ligh...,ehjcu0g,aw0ve0,1.551417e+09,It looks like the purchase button has just bee...,2019-03-01,3


In [110]:
Bolt['time'] = Bolt['time'].apply(lambda x: x.split(' ')[0])
Leaf['time'] = Leaf['time'].apply(lambda x: x.split(' ')[0])
Tesla['time'] = Tesla['time'].apply(lambda x: x.split(' ')[0])

In [111]:
Bolt['month'] = Bolt['time'].apply(lambda x: x.split('-')[1])
Leaf['month'] = Leaf['time'].apply(lambda x: x.split('-')[1])
Tesla['month'] = Tesla['time'].apply(lambda x: x.split('-')[1])

In [112]:
Leaf['comment']

0       Is there a huge difference in price? Of course...
1       The price is almost same. 2016 is $100 cheaper...
2       No OP, but mind if I ask why you'd set that pr...
3       Does the 2017 have any of the upgraded package...
4       I was quoting private seller prices, with a ba...
                              ...                        
5959    I've got a 2018 LEAF, and while I've never had...
5960    I don’t think V2G is as important as we origin...
5961    I thought you meant how chademo was originally...
5962    Yeah, my local mall has 2 evgo stations with c...
5963    Load control wouldn't have anything to do with...
Name: comment, Length: 5964, dtype: object

In [113]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [114]:
def transfer(data):
    data = data

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in         allowed_postags]))
        return texts_out

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # Run in terminal: python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return data_lemmatized


In [115]:
Leaf.comment = transfer(Leaf.comment.values.tolist())

In [116]:
Bolt.comment = transfer(Bolt.comment.values.tolist())

In [ ]:
Tesla.comment = transfer(Tesla.comment.values.tolist())

In [195]:
Bolt.head()

,comment,comment_id,post_id,post_time,post_title,time,month
0,precondition charge stat mileage all free year...,epkew7k,bv20re,1.559270e+09,Is remote start/precondition using MyChevrolet...,2019-05-31,05
1,onstar service precondition app work lock unlo...,epk9lrk,bv20re,1.559270e+09,Is remote start/precondition using MyChevrolet...,2019-05-31,05
2,cheat handy way use alexa basic connection con...,epke6ad,bv20re,1.559270e+09,Is remote start/precondition using MyChevrolet...,2019-05-31,05
3,original bolt year basic onstar include other ...,epkv369,bv20re,1.559270e+09,Is remote start/precondition using MyChevrolet...,2019-05-31,05
4,mine work only when bluetooth range cancel onstar,epk8zjl,bv20re,1.559270e+09,Is remote start/precondition using MyChevrolet...,2019-05-31,05


In [156]:
Tesla.head(1000)

,comment,comment_id,post_id,post_time,post_title,time,month
0,reminder include price wall connector include ...,epkqq0x,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
1,blue performance model confirm,epkl3uo,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
2,live too base same option same delivery estima...,epklx56,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
3,congrat come pretty soon too sadly can fork ri...,epklo12,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
4,standard range autopilot come well luxury car ...,epklvos,bv2t27,1.559275e+09,Australian model 3 Configurator is live!,2019-05-31,5
...,...,...,...,...,...,...,...
995,notice too last time,epjwrq2,buxg10,1.559245e+09,A Model S coupe in this Los Angeles Airport ad...,2019-05-31,5
996,man people really try too hard,epj40vi,buxg10,1.559245e+09,A Model S coupe in this Los Angeles Airport ad...,2019-05-30,5
997,try hard enough,epjm3ox,buxg10,1.559245e+09,A Model S coupe in this Los Angeles Airport ad...,2019-05-30,5
998,very few fact base youtube video battery base ...,epkwzok,bux9gt,1.559244e+09,"How Tesla batteries work, Sean Mitchell with P...",2019-05-31,5


In [182]:
Tesla.month = Tesla.month.apply(str)

In [192]:
Tesla['month']= Tesla['month'].astype(str)

In [ ]:
Topic: CEO

[(8,
  [('elon', 0.04012874658471498),
   ('musk', 0.0160751385435088),
   ('state', 0.01253824437301959),
   ('tweet', 0.011551732759913988),
   ('fact', 0.009828747039328626),
   ('law', 0.009737824310010138),
   ('sec', 0.009610532488964254),
   ('claim', 0.00939231793859988),
   ('public', 0.00936049498333841),
   ('word', 0.008869512245018571)]),

Topic: Truck production

 (11,
  [('tesla', 0.034165642136486936),
   ('make', 0.030916417852194445),
   ('truck', 0.028833206486739652),
   ('big', 0.024993657670315944),
   ('production', 0.02241281736041996),
   ('build', 0.021398044610971254),
   ('start', 0.02133950002927229),
   ('line', 0.021061413266202214),
   ('design', 0.01714380500751322),
   ('end', 0.01449954140077669)]),
 Topic: Energy efficiency

 (10,
  [('battery', 0.05482402086063075),
   ('mile', 0.05154616884829772),
   ('range', 0.04227045306141864),
   ('power', 0.028229311261886537),
   ('high', 0.01578817638130327),
   ('gas', 0.015433580253320015),
   ('pack', 0.013409787718000943),
   ('low', 0.012990326444654896),
   ('rate', 0.012986002101630709),
   ('energy', 0.012302755903809314)]),
Topic: Driver experience

 ((12,
  [('change', 0.045256801788868296),
   ('lane', 0.04209843996067274),
   ('road', 0.03150966215099232),
   ('stop', 0.030104495662690436),
   ('driver', 0.029669459598200693),
   ('speed', 0.023696414432756475),
   ('turn', 0.02336578702374427),
   ('brake', 0.018214960020185675),
   ('traffic', 0.01758850808732044),
   ('move', 0.017562405923451055)])



In [198]:
def select(dataframe):
    df = dataframe
    df = df[df.month == '5' ] ### Select the month here !
    # Cost
    criteria=['elon','musk','state','tweet','fact','law','sec','claim','public','word'] ### Change the topic word here !!! 
    new=[]
    for i in df['comment']:
        
        if any(ext in i for ext in criteria):
            
            new.append(i)
    sentiment_scores = [af.score(article) for article in new]
    sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]
    df = pd.DataFrame([new, sentiment_scores, sentiment_category]).T
    df.columns = ['comment', 'sentiment_score', 'sentiment_category']
    df['sentiment_score'] = df.sentiment_score.astype('float')
    return df



In [69]:
Leaf_3 = select(Leaf)

In [71]:
Leaf_4 = select(Leaf)

In [73]:
Leaf_5 = select(Leaf)

In [89]:
Bolt_3_Dexp = select(Bolt)

In [91]:
Bolt_4_Dexp = select(Bolt)

In [93]:
Bolt_5_Dexp = select(Bolt)

In [95]:
Bolt_3_Ser = select(Bolt)

In [97]:
Bolt_4_Ser = select(Bolt)

In [99]:
Bolt_5_Ser = select(Bolt)

In [101]:
Bolt_3_Enef = select(Bolt)

In [103]:
Bolt_4_Enef = select(Bolt)

In [106]:
Bolt_5_Enef = select(Bolt)

In [199]:
Tesla_5_CEO = select(Tesla)

In [201]:
Tesla_5_CEO

,comment,sentiment_score,sentiment_category


In [119]:
print(Bolt_3_Dexp.sentiment_score.mean(),
Bolt_4_Dexp.sentiment_score.mean(),
Bolt_5_Dexp.sentiment_score.mean())

2.432065217391304 1.3100436681222707 0.7402597402597403


In [123]:
Bolt_3_Dexp.to_csv('Bolt_3_Dexp.csv',index=False)
Bolt_4_Dexp.to_csv('Bolt_4_Dexp.csv',index=False)
Bolt_5_Dexp.to_csv('Bolt_5_Dexp.csv',index=False)
Bolt_3_Ser.to_csv('Bolt_3_Ser.csv',index=False)
Bolt_4_Ser.to_csv('Bolt_4_Ser.csv',index=False)
Bolt_5_Ser.to_csv('Bolt_5_Ser.csv',index=False)
Bolt_3_Enef.to_csv('Bolt_3_Enef.csv',index=False)
Bolt_4_Enef.to_csv('Bolt_4_Enef.csv',index=False)
Bolt_5_Enef.to_csv('Bolt_5_Enef.csv',index=False)


In [126]:
bolt_enef = [Bolt_3_Enef.sentiment_score.mean(),
Bolt_4_Enef.sentiment_score.mean(),
Bolt_5_Enef.sentiment_score.mean()]
bolt_dexp = [Bolt_3_Dexp.sentiment_score.mean(),
Bolt_4_Dexp.sentiment_score.mean(),
Bolt_5_Dexp.sentiment_score.mean()]
bolt_ser = [Bolt_3_Ser.sentiment_score.mean(),
Bolt_4_Ser.sentiment_score.mean(),
Bolt_5_Ser.sentiment_score.mean()]

In [127]:
bolt = []
bolt.append(bolt_dexp)
bolt.append(bolt_ser)
bolt.append(bolt_enef)


In [129]:
dbolt = pd.DataFrame(bolt)
dbolt.columns = ['Mar','Apr','May']

In [131]:
dbolt.rename(index={0:'Driver Experience',1:'Service',2:'Energy Efficiency'},inplace=True)

In [133]:
dbolt.to_csv('Bolt_sentiment.csv', index = True)

In [120]:
print(Bolt_3_Ser.sentiment_score.mean(),
Bolt_4_Ser.sentiment_score.mean(),
Bolt_5_Ser.sentiment_score.mean())

2.6297520661157026 1.7560975609756098 1.3062330623306233


In [125]:
print(Bolt_3_Enef.sentiment_score.mean(),
Bolt_4_Enef.sentiment_score.mean(),
Bolt_5_Enef.sentiment_score.mean())

2.6106623586429727 1.9896640826873384 1.4195121951219511


In [122]:
Tesla_3_CEO.sentiment_score.mean()

1.0666613760812633

In [88]:
Bolt_3 = Bolt[Bolt.month == '03' ]
Bolt_4 = Bolt[Bolt.month == '04' ]
Bolt_5 = Bolt[Bolt.month == '05' ]
print(len(Bolt_3))
print(len(Bolt_4))
print(len(Bolt_5))


1864
1133
1209


In [34]:
Leaf_3 = Leaf[Leaf.month == '03' ]
Leaf_4 = Leaf[Leaf.month == '04' ]
Leaf_5 = Leaf[Leaf.month == '05' ]
print(len(Leaf_3))
print(len(Leaf_4))
print(len(Leaf_5))

AttributeError: 'DataFrame' object has no attribute 'month'

In [22]:
Tesla_3 = Tesla[Tesla.month == '03' ]
Tesla_4 = Tesla[Tesla.month == '04' ]
Tesla_5 = Tesla[Tesla.month == '05' ]
print(len(Tesla_3))
print(len(Tesla_4))
print(len(Tesla_5))

3215
4004
3928


In [76]:
Leaf_3.to_csv('Leaf_3_2.csv',index=False)

In [78]:
Leaf_3.sentiment_score.mean()

1.1727272727272726

In [75]:
Leaf_4.to_csv('Leaf_4_2.csv',index=False)

In [79]:
Leaf_4.sentiment_score.mean()

0.8267326732673267

In [74]:
Leaf_5.to_csv('Leaf_5_2.csv',index=False)

In [80]:
Leaf_5.sentiment_score.mean()

1.5340314136125655

In [124]:
Leaf3_1 = pd.read_csv('Leaf_3.csv',sep=',')
Leaf4_1 = pd.read_csv('Leaf_4.csv',sep=',')
Leaf5_1 = pd.read_csv('Leaf_5.csv',sep=',')

Leaf3_1.to_csv('Leaf_3_Per.csv',index=False)
Leaf4_1.to_csv('Leaf_4_Per.csv',index=False)
Leaf5_1.to_csv('Leaf_5_Per.csv',index=False)
Leaf3_2.to_csv('Leaf_3_Wor.csv',index=False)
Leaf4_2.to_csv('Leaf_4_Wor.csv',index=False)
Leaf5_2.to_csv('Leaf_5_Wor.csv',index=False)
Leaf_3.to_csv('Leaf_3_Ext.csv',index=False)
Leaf_4.to_csv('Leaf_4_Ext.csv',index=False)
Leaf_5.to_csv('Leaf_5_Ext.csv',index=False)

In [134]:
Leaf_per = [Leaf3_1.sentiment_score.mean(),
Leaf4_1.sentiment_score.mean(),
Leaf5_1.sentiment_score.mean()]
Leaf_wor = [Leaf3_2.sentiment_score.mean(),
Leaf4_2.sentiment_score.mean(),
Leaf5_2.sentiment_score.mean()]
Leaf_ext = [Leaf_3.sentiment_score.mean(),
Leaf_4.sentiment_score.mean(),
Leaf_5.sentiment_score.mean()]

In [135]:
leaf = []
leaf.append(Leaf_per)
leaf.append(Leaf_wor)
leaf.append(Leaf_ext)

In [137]:
dleaf = pd.DataFrame(leaf)
dleaf.columns = ['Mar','Apr','May']

In [138]:
dleaf.rename(index={0:'Car Performance/ Engine problem',1:'Worth/ Value',2:'Exterior Design'},inplace=True)

In [139]:
dleaf.to_csv('Leaf_sentiment.csv', index = True)